In [ ]:
import datetime
import requests
import pandas as pd
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')


In [ ]:
SAVE_IMAGES = True

def plot(axes, title=None, file=None, ylim=None, pppp=plt, xticks=None):
    if title: axes.set_title(title)
    axes.legend(loc="upper left")
    axes.set_xlim(None, None)
    axes.autoscale()
    pppp.tight_layout()
    if ylim: 
        if type(ylim) == int: pppp.ylim(ylim)
        else: pppp.ylim(ylim[0], ylim[1])
    # print(f"ylim={ylim} type={type(ylim)} int={type(ylim)==int}")
    if xticks: pppp.xticks(xticks, rotation=90)
    if SAVE_IMAGES and file: pppp.savefig(file)
    return pppp

def f(valor, plus=False):
    if valor is None: return None
    valor = valor if type(valor) == int else float(valor)
    r = format(valor, ",").replace(".","!").replace(",",".").replace("!",",")
    return f"+{r}" if plus and valor > 0 else r


In [ ]:
# read data and reindex by date, for plots
data = pd.read_csv(Path.cwd() / '..' / '..' / 'data.csv', parse_dates=['data'], index_col='data', dayfirst=True)
data.tail(1).index


In [ ]:
ARS = ['arsnorte', 'arscentro', 'arslvt', 'arsalentejo', 'arsalgarve', 'acores', 'madeira']
IDADES = ['0_9', '10_19', '20_29', '30_39', '40_49', '50_59', '60_69', '70_79', '80_plus']


In [ ]:
df = data
for j in ['confirmados', 'obitos']:
    df[f'{j}_7'] = df[f'{j}'].diff(7).div(7)
    for k in ARS:
        df[f'{j}_{k}_7'] = df[f'{j}_{k}'].diff(7).div(7)
    for k in IDADES:
        df[f'{j}_{k}'] = df[f'{j}_{k}_f'] + df[f'{j}_{k}_m']
        df[f'{j}_{k}_7'] = df[f'{j}_{k}'].diff(7).div(7)



In [ ]:
ymax = { 
    'confirmados': df['confirmados_7'].max(), 'obitos': df['obitos_7'].max(),
    'internados_enfermaria': df['internados_enfermaria'].max(), 'internados_uci': df['internados_uci'].max(),
}
for j in ['confirmados', 'obitos']:
    df[f'{j}_7A'] = df[f'{j}_7'].div(ymax[j])
    for k in ARS:
        df[f'{j}_{k}_7A'] = df[f'{j}_{k}_7'].div(ymax[j])
    for k in IDADES:
        df[f'{j}_{k}_7A'] = df[f'{j}_{k}_7'].div(ymax[j])
    for k in ['enfermaria', 'uci']:
        df[f'internados_{k}_A'] = df[f'internados_{k}'].div(ymax[f'internados_{k}'])


In [ ]:
c7 = df[ ['confirmados_7A', 'obitos_7A', 'internados_enfermaria_A', 'internados_uci_A'] ].copy()
c7.index = c7.index + pd.DateOffset(days= -260)
cols = ['confirmados_7A2', 'obitos_7A2', 'internados_enfermaria_A2', 'internados_uci_A2']
c7.columns = cols
for col in cols: 
    if col in df: df.drop(col, axis=1, inplace=True)
df = pd.merge(df, c7, how="left", on="data")


In [ ]:
fix, axes = plt.subplots(figsize=(20,10))
C= {
    'confirmados_7A': ('#AFA', '#3F3'),
    'obitos_7A': ('#F00', '#F33'),
    'internados_enfermaria_A': ('#00F', '#33F'),
    'internados_uci_A': ('#FF0', '#FF3'),
}
for k in ['confirmados_7A', 'obitos_7A', 'internados_enfermaria_A', 'internados_uci_A']:
    axes.plot(df[k], lw=4, alpha=0.5, label=k, color=C[k][0] )
for k in ['confirmados_7A', 'obitos_7A', 'internados_enfermaria_A', 'internados_uci_A']:
    axes.plot(df[f"{k}2"], lw=4, alpha=1, label=f"{k}2", color=C[k][1] )
#axes.plot(df['foo'], lw=2, alpha=1, label='CCC')
plot(axes, title="Confirmados vs Enfermaria vs UCI vs Obitos, ajustados ao pico respectivo", ylim=(0, None)).show()


In [ ]:
for k in ARS:
    #data[[f'confirmados_{k}_7A', f'obitos_{k}_7A']].plot(figsize=(20,5), ylim=(0, None), lw=2)
    print()
    

In [ ]:
for k in IDADES:
    data[f'confirmados_{k}_7B'] = data[f'confirmados_{k}_7'].div( data[f'confirmados_{k}_7'].max() )
    data[f'obitos_{k}_7B'] = data[f'obitos_{k}_7'].div( data[f'obitos_{k}_7'].max() )
    data[[f'confirmados_{k}_7B', f'obitos_{k}_7B']].plot(figsize=(20,5), ylim=(0, None), lw=2)


In [ ]:
#cols = [f'confirmados_{k}_7B' for k in IDADES] + [f'obitos_{k}_7B' for k in IDADES]

cols = [f'confirmados_{k}_7B' for k in IDADES]
data[-90:][cols].plot(figsize=(20,10), ylim=(0, None), lw=2)

cols = [f'obitos_{k}_7B' for k in IDADES]
data[-90:][cols].plot(figsize=(20,10), ylim=(0, None), lw=2)


In [ ]:

#df[ ['confirmados', 'obitos', 'ativos', 'recuperados'] ].plot(figsize=(20,5), ylim=(0,None))
df[ ['ativos'] ].plot(figsize=(20,5), ylim=(0,None))


In [ ]:
df['ativos']